In [1]:
# Necessary modules
import pandas as pd
import numpy  as np

from keras.models     import Sequential
from keras.layers     import Dense, Dropout, Activation, Flatten
from keras.layers     import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [32]:
# Get the data
datas = []
for fold in range(5):
    path = "../data/face_image_project/fold_%d_data.txt" % (fold)
    data_fold = pd.read_csv(filepath_or_buffer=path, sep="\t")
    data_fold = data_fold.sample(frac=1).reset_index(drop=True)
    datas.append(data_fold)

In [33]:
data = pd.concat(datas, ignore_index=True)
data.count()
(data.age == 'None').sum()

748

In [34]:
data = data[["user_id","original_image","face_id","gender", "age"]]
data.head()

,user_id,original_image,face_id,gender,age
0,20254529@N04,9981772873_b89d389e58_o.jpg,51,m,"(38, 43)"
1,10113099@N03,12038644665_3de8d52d98_o.jpg,448,f,"(8, 12)"
2,62501130@N02,11399554613_8f5b43bbdd_o.jpg,99,m,"(25, 32)"
3,29671106@N00,8137378789_7223f571bf_o.jpg,199,f,"(38, 43)"
4,9017386@N06,10770684434_d85338d851_o.jpg,219,f,"(25, 32)"


In [35]:
data = data[data.gender != "u"]
data.gender.value_counts()

f    9372
m    8120
Name: gender, dtype: int64

In [36]:
data.loc[data.gender == "f", "gender"] = 0
data.loc[data.gender == "m", "gender"] = 1

In [37]:
path_template = "../data/face_image_project/aligned/%s/landmark_aligned_face.%d.%s"
data["file_path"] = data[["user_id","face_id","original_image"]].apply(lambda x:  
                                                                   path_template % (x[0],x[1],x[2]),
                                                                   axis=1)

In [38]:
## Drop the None data
data = data[data.age != 'None']

In [57]:
# Building a vainilla CNN
vainilla_cnn = Sequential()
vainilla_cnn.add(Convolution2D(32, 3, 3, input_shape=(227,227,3)))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(MaxPooling2D(pool_size=(2, 2)))
vainilla_cnn.add(Convolution2D(64, 3, 3))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(Convolution2D(64, 3, 3))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(MaxPooling2D(pool_size=(2, 2)))
vainilla_cnn.add(Flatten())
vainilla_cnn.add(Dense(output_dim=200, input_dim=500))
vainilla_cnn.add(BatchNormalization())
vainilla_cnn.add(Activation("relu"))
vainilla_cnn.add(Dense(output_dim=100, input_dim=200))
vainilla_cnn.add(BatchNormalization())
vainilla_cnn.add(Activation("relu"))
vainilla_cnn.add(Dense(output_dim=29))
vainilla_cnn.add(Activation("softmax"))

In [59]:
vainilla_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
data = data[["file_path","gender", "age"]]
data = pd.get_dummies(data, columns=["age"])

In [60]:
data.columns

Index([u'file_path', u'gender', u'age_(0, 2)', u'age_(15, 20)',
       u'age_(25, 32)', u'age_(27, 32)', u'age_(38, 42)', u'age_(38, 43)',
       u'age_(38, 48)', u'age_(4, 6)', u'age_(48, 53)', u'age_(60, 100)',
       u'age_(8, 12)', u'age_(8, 23)', u'age_13', u'age_2', u'age_22',
       u'age_23', u'age_29', u'age_3', u'age_32', u'age_34', u'age_35',
       u'age_36', u'age_42', u'age_45', u'age_46', u'age_55', u'age_56',
       u'age_57', u'age_58'],
      dtype='object')

In [61]:
age_cols = [u'age_(0, 2)', u'age_(15, 20)',
       u'age_(25, 32)', u'age_(27, 32)', u'age_(38, 42)', u'age_(38, 43)',
       u'age_(38, 48)', u'age_(4, 6)', u'age_(48, 53)', u'age_(60, 100)',
       u'age_(8, 12)', u'age_(8, 23)', u'age_13', u'age_2', u'age_22',
       u'age_23', u'age_29', u'age_3', u'age_32', u'age_34', u'age_35',
       u'age_36', u'age_42', u'age_45', u'age_46', u'age_55', u'age_56',
       u'age_57', u'age_58']

In [62]:
import scipy.misc

i_width  = 227
i_height = 227

def train_generator(data):
    while True:
        start, end = 0, 32
        while end < len(data):
            data = data.sample(frac=1).reset_index(drop=True)
            sample  = data[start:end]

            X = pd.DataFrame(sample["file_path"].apply(lambda x:  img_to_array( scipy.misc.imresize(load_img(x), (i_height, i_width) ) ) ))           
            X = X["file_path"].apply(lambda x: x.reshape((1,)+ x.shape))
            X = np.vstack(X)

            Y = sample[age_cols].as_matrix()
            
            yield (X, Y)
            start += 32
            end += 32

In [63]:
a,b = train_generator(data).next()
print a.shape
print b.shape

(32, 227, 227, 3)
(32, 29)


In [64]:
history = vainilla_cnn.fit_generator(train_generator(data), samples_per_epoch=64, nb_epoch=2000)

Epoch 1/2000
64/64 [==============================] - 61s - loss: 3.7861 - acc: 0.0938     
Epoch 2/2000
64/64 [==============================] - 1s - loss: 3.4181 - acc: 0.0781     
Epoch 3/2000
64/64 [==============================] - 1s - loss: 3.1024 - acc: 0.1250     
Epoch 4/2000
64/64 [==============================] - 1s - loss: 3.1511 - acc: 0.1250     
Epoch 5/2000
64/64 [==============================] - 1s - loss: 3.0335 - acc: 0.2188     
Epoch 6/2000
64/64 [==============================] - 1s - loss: 3.0082 - acc: 0.1719     

In [65]:
vainilla_cnn.save_weights('vainilla_cnn_age_1.h5')